In [ ]:
import os
import json
from kaggle_secrets import UserSecretsClient
from openai import OpenAI

# Read the API key from Kaggle secrets
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("OPENAI_API_KEY")  # must match your secret name

if not api_key:
    raise RuntimeError(
        "Could not read OPENAI_API_KEY from Kaggle secrets. "
    )

# Create the OpenAI client (v1-style)
client = OpenAI(api_key=api_key)

MODEL_NAME = "gpt-4o-mini"  # or another chat-capable model available to your account

SYSTEM_PROMPT = (
    "You are an AI agent that first analyzes the user's message and then replies.\n"
    "1. Decide the high-level intent of the user in a few words.\n"
    "2. Estimate the overall sentiment as one of: 'positive', 'neutral', or 'negative'.\n"
    "3. Write a clear, helpful reply.\n\n"
    "Return your answer strictly as JSON with the keys:\n"
    "{\n"
    '  \"intent\": \"<short_intent>\",\n'
    '  \"sentiment\": \"<positive|neutral|negative>\",\n'
    " \"reply\": \"<your natural language response>\"\n"
    "}\n"
)

# 2. Core agent function (new API)

def analyze_and_respond(user_message: str) -> dict:
    """
    Send the user's message to the OpenAI chat model and return:
    - intent (str)
    - sentiment (str)
    - reply (str)
    """
    completion = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_message},
        ],
        temperature=0.7,
        max_tokens=300,
    )

    raw_content = completion.choices[0].message.content

    try:
        data = json.loads(raw_content)
    except json.JSONDecodeError:
        data = {
            "intent": "unknown",
            "sentiment": "neutral",
            "reply": raw_content.strip(),
        }

    return {
        "intent": str(data.get("intent", "unknown")),
        "sentiment": str(data.get("sentiment", "neutral")),
        "reply": str(data.get("reply", raw_content)).strip(),
    }

# 3. Simple interactive loop (unchanged)

def chat_loop() -> None:
    """
    Run an interactive console loop where the user can chat with the agent.
    """
    print("=== Simple AI Agent (intent + sentiment + reply) ===")

    while True:
        user_message = input("You: ").strip()
        if not user_message:
            continue

        if user_message.lower() in {"exit", "quit", "q"}:
            print("Agent: Goodbye!")
            break

        analysis = analyze_and_respond(user_message)

        intent = analysis["intent"]
        sentiment = analysis["sentiment"]
        reply = analysis["reply"]

        print(f"[Intent: {intent} | Sentiment: {sentiment}]")
        print(f"Agent: {reply}\n")


if __name__ == "__main__":
    chat_loop()

# Simple AI Agent using LangChain (RunnableLambda), rule-based analysis, and templated replies.

In [ ]:
!pip install -q "protobuf<6" --quiet
!pip install "langchain_huggingface" --quiet

In [2]:
import re
from typing import Dict

from langchain_core.runnables import RunnableLambda


# ---------------------------------------------------------------------
# 1. Lightweight intent + sentiment analysis
# ---------------------------------------------------------------------

def detect_intent(text: str) -> str:
    """Very simple rule-based intent classifier."""
    lowered = text.lower()

    if any(word in lowered for word in ["hi", "hello", "hey", "good morning", "good evening"]):
        return "greeting"

    if any(phrase in lowered for phrase in ["thank you", "thanks", "appreciate it"]):
        return "gratitude"

    if any(word in lowered for word in ["help", "how do i", "how to", "can you", "could you", "what can you"]):
        return "information_request"

    if any(word in lowered for word in ["bye", "goodbye", "see you", "later"]):
        return "farewell"

    # Fallback if nothing obvious matched
    return "general_conversation"


def analyze_sentiment(text: str) -> str:
    """
    Tiny keyword-based sentiment estimator.

    Counts positive/negative words and compares their scores.
    This is deliberately simple, but enough to demonstrate analysis.
    """
    lowered = text.lower()

    positive_words = [
        "good", "great", "awesome", "love", "like",
        "happy", "nice", "amazing", "fantastic", "cool",
    ]
    negative_words = [
        "bad", "terrible", "hate", "dislike",
        "sad", "angry", "upset", "worried",
        "annoyed", "frustrated",
    ]

    pos_score = sum(1 for w in positive_words if re.search(rf"\b{re.escape(w)}\b", lowered))
    neg_score = sum(1 for w in negative_words if re.search(rf"\b{re.escape(w)}\b", lowered))

    if pos_score > neg_score:
        return "positive"
    if neg_score > pos_score:
        return "negative"
    return "neutral"


# ---------------------------------------------------------------------
# 2. Analysis + reply functions (pure Python)
# ---------------------------------------------------------------------

def analyze_message(user_text: str) -> Dict[str, str]:
    """
    Step 1: Given the raw user text, compute intent and sentiment.
    This is the first stage in the LangChain pipeline.
    """
    intent = detect_intent(user_text)
    sentiment = analyze_sentiment(user_text)

    return {
        "text": user_text,
        "intent": intent,
        "sentiment": sentiment,
    }


def craft_reply(analysis: Dict[str, str]) -> Dict[str, str]:
    """
    Step 2: Given the analysis dict, generate a short, clear reply.
    This is deterministic and rule-based (no external model).
    """
    text = analysis["text"]
    intent = analysis["intent"]
    sentiment = analysis["sentiment"]

    # Base reply per intent
    if intent == "greeting":
        reply = "Hello! I am a simple AI agent here to chat with you and answer basic questions."
    elif intent == "gratitude":
        reply = "You are welcome; glad I could help."
    elif intent == "information_request":
        reply = "I can explain what I am, what I can do, or help you think through simple questions step by step."
    elif intent == "farewell":
        reply = "Goodbye! It was nice talking with you."
    else:
        reply = "I am an AI agent that analyzes your message and replies with a short, helpful response."

    # Very light sentiment-aware adjustment
    if sentiment == "negative" and intent not in {"farewell", "gratitude"}:
        reply += " I am sorry that things feel difficult; I will try to respond in a calm and supportive way."
    elif sentiment == "positive" and intent not in {"farewell", "gratitude"}:
        reply += " It sounds like you are feeling positive, which is great."

    # Add original text to the dict in case you want to log it later
    return {
        "text": text,
        "intent": intent,
        "sentiment": sentiment,
        "reply": reply,
    }


# ---------------------------------------------------------------------
# 3. Build a LangChain pipeline from these functions
# ---------------------------------------------------------------------

# Wrap the plain Python functions as LangChain Runnables
analysis_runnable = RunnableLambda(analyze_message)
reply_runnable = RunnableLambda(craft_reply)

# Chain them: user_text (str) -> analysis dict -> dict with reply
agent_chain = analysis_runnable | reply_runnable


# ---------------------------------------------------------------------
# 4. Simple console chat loop
# ---------------------------------------------------------------------

def chat_loop() -> None:
    """
    Run a text-based chat loop.

    Type 'exit', 'quit', or 'q' to stop.
    """
    print("=== Simple LangChain AI Agent (rule-based) ===")
    print("Type 'exit', 'quit', or 'q' to end the conversation.\n")

    while True:
        user_text = input("You: ").strip()
        if not user_text:
            continue

        if user_text.lower() in {"exit", "quit", "q"}:
            print("Agent: Goodbye!")
            break

        # Use the LangChain pipeline to process the message
        result = agent_chain.invoke(user_text)

        print(f"[Intent: {result['intent']} | Sentiment: {result['sentiment']}]")
        print(f"Agent: {result['reply']}\n")


# Only run the loop if this file / cell is executed directly
if __name__ == "__main__":
    chat_loop()

=== Simple LangChain AI Agent (rule-based) ===
Type 'exit', 'quit', or 'q' to end the conversation.



You:  Hi


[Intent: greeting | Sentiment: neutral]
Agent: Hello! I am a simple AI agent here to chat with you and answer basic questions.



You:  What can you do


[Intent: information_request | Sentiment: neutral]
Agent: I can explain what I am, what I can do, or help you think through simple questions step by step.



...

[Intent: general_conversation | Sentiment: neutral]
Agent: I am an AI agent that analyzes your message and replies with a short, helpful response.



You:  Can you write code?


[Intent: information_request | Sentiment: neutral]
Agent: I can explain what I am, what I can do, or help you think through simple questions step by step.



You:  q


Agent: Goodbye!
